# Preproceso para csvs

En la carpeta datos tenemos un csv para cada mes del año, vamos leer y procesarlos para obtaner un archivo con todos los tuitas del año para la zona de la CDMX.

Como son un montón de datos, entonces mejor vamos a cortar cada mes con la zona de análisis para no cargar con todos los tuits que njo nos interesan

In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
from shapely.geometry import LineString, Point
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import multiprocessing
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.preprocess import preprocesa_archivo

Empezamos por leer un sólo archivo y ver cómo hacemos todo el preproceso

In [3]:
un_mes = pd.read_csv("../data/enero2018.csv")
un_mes

,ID,Usuario,Fecha_tweet,Texto,Latitud,Longitud,Sentimiento,Idioma
0,947618373680103424,LuisDarioGarcia,2018-01-01T00:00:00.000Z,"De forma general, el 2017 estuvo muy culero y ...",19.432722,-99.017016,negativo,Español
1,947618374309302272,_BlackBeauty12_,2018-01-01T00:00:00.000Z,I Swear!! We Was Always In The Room Talking Ab...,32.576227,-86.680737,positivo,Inglés
2,947618370790350848,Mia_soga,2018-01-01T00:00:00.000Z,@fedevigevani https://t.co/ppuinwo7Wb,19.128538,-104.363380,neutro,Indeterminado
3,947618373239738369,johnpneedham,2018-01-01T00:00:00.000Z,Chicken &amp; corn chili with a cornbread topp...,30.054100,-95.387000,neutro,Inglés
4,947618370958057472,basic_micah,2018-01-01T00:00:00.000Z,Idk if what I’m sensing is right...I don’t wan...,30.521057,-87.202962,neutro,Inglés
...,...,...,...,...,...,...,...,...
8256469,958852390257479680,banditchinorich,2018-01-31T23:59:59.000Z,What is was and what happen,29.838495,-95.446487,neutro,Inglés
8256470,958852391369039872,_ArlingtonTX,2018-01-31T23:59:59.000Z,Arlington: 6:01pm: sunset,32.740000,-97.110000,neutro,Noruego
8256471,958852393768275969,tews2com,2018-01-31T23:59:59.000Z,"Wind 9.6 mph S. Barometer 29.863 in, Falling s...",32.424167,-96.952222,neutro,Inglés
8256472,958852393176793090,juanmapregunta,2018-01-31T23:59:59.000Z,"#Entérate Ministros de México, #EEUU y #Canadá...",23.625418,-102.558037,positivo,Español


Nos quedamos con las columnas que nos interesan y convertimos en GeoDataFrame

In [4]:
un_mes = un_mes.loc[:,['ID', 'Usuario', 'Fecha_tweet', 'Latitud', 'Longitud']]
un_mes = (gpd.GeoDataFrame(un_mes, geometry=gpd.points_from_xy                                          (un_mes.Longitud, un_mes.Latitud))
          .drop(['Latitud', 'Longitud'], axis=1)
          .set_crs(4326)
          )
un_mes

,ID,Usuario,Fecha_tweet,geometry
0,947618373680103424,LuisDarioGarcia,2018-01-01T00:00:00.000Z,POINT (-99.01702 19.43272)
1,947618374309302272,_BlackBeauty12_,2018-01-01T00:00:00.000Z,POINT (-86.68074 32.57623)
2,947618370790350848,Mia_soga,2018-01-01T00:00:00.000Z,POINT (-104.36338 19.12854)
3,947618373239738369,johnpneedham,2018-01-01T00:00:00.000Z,POINT (-95.38700 30.05410)
4,947618370958057472,basic_micah,2018-01-01T00:00:00.000Z,POINT (-87.20296 30.52106)
...,...,...,...,...
8256469,958852390257479680,banditchinorich,2018-01-31T23:59:59.000Z,POINT (-95.44649 29.83850)
8256470,958852391369039872,_ArlingtonTX,2018-01-31T23:59:59.000Z,POINT (-97.11000 32.74000)
8256471,958852393768275969,tews2com,2018-01-31T23:59:59.000Z,POINT (-96.95222 32.42417)
8256472,958852393176793090,juanmapregunta,2018-01-31T23:59:59.000Z,POINT (-102.55804 23.62542)


Filtramos con los distritos de análisis (y de una vez asignamos el id del distrito) 

In [7]:
distritos = gpd.read_file("../data/shapes/DistritosEODHogaresZMVM2017.shp")
un_mes = (gpd.sjoin(un_mes, distritos.to_crs(4326))
          .drop(['index_right', 'Descripcio'], axis=1)
          )
un_mes

,ID,Usuario,Fecha_tweet,geometry,Distrito
0,947618373680103424,LuisDarioGarcia,2018-01-01T00:00:00.000Z,POINT (-99.01702 19.43272),173
110,947618450163281921,venus_saiyan,2018-01-01T00:00:19.000Z,POINT (-99.01702 19.43272),173
115,947618453179043840,survivorcancer,2018-01-01T00:00:19.000Z,POINT (-99.01702 19.43272),173
197,947618513140768774,LuisDarioGarcia,2018-01-01T00:00:34.000Z,POINT (-99.01702 19.43272),173
643,947618841403756546,LuisDarioGarcia,2018-01-01T00:01:52.000Z,POINT (-99.01702 19.43272),173
...,...,...,...,...,...
5478791,955227268078624773,dan_mcruz,2018-01-21T23:55:02.000Z,POINT (-98.98707 19.40039),183
6767545,956924689179402241,OskrWar,2018-01-26T16:19:59.000Z,POINT (-98.98619 19.40337),183
7176608,957445347118403584,arhx3,2018-01-28T02:48:53.000Z,POINT (-98.99752 19.39815),183
5725275,955550568876974080,Guiller18152066,2018-01-22T21:19:43.000Z,POINT (-99.07022 19.57566),148


Ahora convertimos `Fecha_tweet` en timestamp para poder manipularlo más adelante

In [8]:
un_mes['fecha_hora_dt'] = pd.to_datetime(
        un_mes['Fecha_tweet'],
        format='%Y-%m-%d %H:%M:%S',
        utc=False)
# un_mes.fecha_hora_dt = un_mes.fecha_hora_dt.dt.tz_convert(
#    'America/Mexico_City')
un_mes

,ID,Usuario,Fecha_tweet,geometry,fecha_hora_dt
0,947618373680103424,LuisDarioGarcia,2018-01-01T00:00:00.000Z,POINT (-99.01702 19.43272),2018-01-01 00:00:00+00:00
1,947618374309302272,_BlackBeauty12_,2018-01-01T00:00:00.000Z,POINT (-86.68074 32.57623),2018-01-01 00:00:00+00:00
2,947618370790350848,Mia_soga,2018-01-01T00:00:00.000Z,POINT (-104.36338 19.12854),2018-01-01 00:00:00+00:00
3,947618373239738369,johnpneedham,2018-01-01T00:00:00.000Z,POINT (-95.38700 30.05410),2018-01-01 00:00:00+00:00
4,947618370958057472,basic_micah,2018-01-01T00:00:00.000Z,POINT (-87.20296 30.52106),2018-01-01 00:00:00+00:00
...,...,...,...,...,...
8256469,958852390257479680,banditchinorich,2018-01-31T23:59:59.000Z,POINT (-95.44649 29.83850),2018-01-31 23:59:59+00:00
8256470,958852391369039872,_ArlingtonTX,2018-01-31T23:59:59.000Z,POINT (-97.11000 32.74000),2018-01-31 23:59:59+00:00
8256471,958852393768275969,tews2com,2018-01-31T23:59:59.000Z,POINT (-96.95222 32.42417),2018-01-31 23:59:59+00:00
8256472,958852393176793090,juanmapregunta,2018-01-31T23:59:59.000Z,POINT (-102.55804 23.62542),2018-01-31 23:59:59+00:00


Este es todo el preprocesamiento que se necesita para cada archivo de datos. Para leer todos los archivos y procesarlos eficientemente vamos a necesitar paralelizar la lectura, para eso necesitamos empaquetar el flujo en una función que tome como entrada el path a cada archivo

In [17]:
def preprocesa_archivo(file_path):
    un_mes = pd.read_csv(file_path)
    distritos = gpd.read_file("../data/shapes/DistritosEODHogaresZMVM2017.shp")
    un_mes = un_mes.loc[:,['ID', 'Usuario', 'Fecha_tweet', 'Latitud', 'Longitud']]
    un_mes = (gpd.GeoDataFrame(un_mes, geometry=gpd.points_from_xy                                          (un_mes.Longitud, un_mes.Latitud))
            .drop(['Latitud', 'Longitud'], axis=1)
            .set_crs(4326)
            )
    un_mes = (gpd.sjoin(un_mes, distritos.to_crs(4326))
        .drop(['index_right', 'Descripcio'], axis=1)
        )
    un_mes['fecha_hora_dt'] = pd.to_datetime(
        un_mes['Fecha_tweet'],
        format='%Y-%m-%d %H:%M:%S',
        utc=False)
    return un_mes

In [18]:
un_mes = preprocesa_archivo("../data/enero2018.csv")

In [19]:
un_mes.head()

,ID,Usuario,Fecha_tweet,geometry,Distrito,fecha_hora_dt
0,947618373680103424,LuisDarioGarcia,2018-01-01T00:00:00.000Z,POINT (-99.01702 19.43272),173,2018-01-01 00:00:00+00:00
110,947618450163281921,venus_saiyan,2018-01-01T00:00:19.000Z,POINT (-99.01702 19.43272),173,2018-01-01 00:00:19+00:00
115,947618453179043840,survivorcancer,2018-01-01T00:00:19.000Z,POINT (-99.01702 19.43272),173,2018-01-01 00:00:19+00:00
197,947618513140768774,LuisDarioGarcia,2018-01-01T00:00:34.000Z,POINT (-99.01702 19.43272),173,2018-01-01 00:00:34+00:00
643,947618841403756546,LuisDarioGarcia,2018-01-01T00:01:52.000Z,POINT (-99.01702 19.43272),173,2018-01-01 00:01:52+00:00


Ya tenemos la función que preprocesa cada archivo, ahora tenemos que sacar todos los archivos con datos

In [2]:
archivos = glob.glob("../data/*.csv")
archivos

['../data/nov_2018.csv',
 '../data/dic_2018.csv',
 '../data/may_2018.csv',
 '../data/sep_2018.csv',
 '../data/enero2018.csv',
 '../data/feb_2018.csv',
 '../data/jun_2018.csv',
 '../data/oct_2018.csv',
 '../data/ago_2018.csv',
 '../data/mar_2018.csv',
 '../data/jul_2018.csv',
 '../data/abr_2018.csv']

Ahora vamos a utilizar `multiprocess.pool` para iterar en paralelo sobre la lista 

In [3]:
a_pool = multiprocessing.Pool()
result = a_pool.map(preprocesa_archivo, archivos)


concatenamos los resulados y exportamos

In [4]:
final = pd.concat(result, axis=0)
final

,ID,Usuario,Fecha_tweet,geometry,Distrito,fecha_hora_dt
2,1057784283908239360,marcojulio93,2018-11-01T00:00:00.000Z,POINT (-98.79725 19.35936),197,2018-11-01 00:00:00+00:00
1093,1057785120684498944,Pontigamex,2018-11-01T00:03:20.000Z,POINT (-98.79725 19.35936),197,2018-11-01 00:03:20+00:00
4358,1057787632682520577,dieghoista,2018-11-01T00:13:19.000Z,POINT (-98.79725 19.35936),197,2018-11-01 00:13:19+00:00
12227,1057793498768965632,XicopataXty,2018-11-01T00:36:37.000Z,POINT (-98.79725 19.35936),197,2018-11-01 00:36:37+00:00
14614,1057795207532609537,KIxtapaluca,2018-11-01T00:43:25.000Z,POINT (-98.79725 19.35936),197,2018-11-01 00:43:25+00:00
...,...,...,...,...,...,...
6628440,988955668005773312,olaga_lidia,2018-04-25T01:39:39.000Z,POINT (-99.06812 19.56454),148,2018-04-25 01:39:39+00:00
6628690,988955854497042432,olaga_lidia,2018-04-25T01:40:24.000Z,POINT (-99.07111 19.56989),148,2018-04-25 01:40:24+00:00
8374195,991099908651454464,mercyrocks,2018-04-30T23:40:06.000Z,POINT (-99.06065 19.57597),148,2018-04-30 23:40:06+00:00
4031099,985360264496336896,Hemer_28,2018-04-15T03:32:48.000Z,POINT (-98.96482 19.44039),187,2018-04-15 03:32:48+00:00


In [5]:
final.to_file("../output/tuits_2018.gpkg", layer='tuits', driver="GPKG")